In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

In [2]:
# Load data
df = pd.read_csv('datacleaned_running_data.csv')

In [3]:
df = df.drop(['Activity ID', 'Activity Name', 'Activity Type', 'Commute', 'Filename', 'Athlete Weight','Activity Date',
                     'Average Grade', 'Commute.1'], axis=1)

In [4]:
# Preprocess data
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

In [5]:
# Prepare sequences for training
seq_length = 30
x = []
y = []
for i in range(len(df) - seq_length - 1):
    x.append(df[i:i + seq_length])
    y.append(df[i + seq_length, 0])
x = np.array(x)
y = np.array(y)

In [6]:
# Prepare train/test split
split_index = int(0.8 * len(x))
x_train, x_test = x[:split_index], x[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [9]:
# Create LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, 17)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [10]:
# Train model
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model.fit(x_train, y_train, epochs=100, batch_size=64, verbose=1, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
5/5 [==============================] - 5s 320ms/step - loss: 0.0059 - val_loss: 0.0048
Epoch 2/100
5/5 [==============================] - 0s 75ms/step - loss: 0.0054 - val_loss: 0.0048
Epoch 3/100
5/5 [==============================] - 0s 41ms/step - loss: 0.0050 - val_loss: 0.0050
Epoch 4/100
5/5 [==============================] - 0s 66ms/step - loss: 0.0051 - val_loss: 0.0048
Epoch 5/100
5/5 [==============================] - 0s 51ms/step - loss: 0.0050 - val_loss: 0.0048
Epoch 6/100
5/5 [==============================] - 1s 195ms/step - loss: 0.0049 - val_loss: 0.0048
Epoch 7/100
5/5 [==============================] - 0s 50ms/step - loss: 0.0049 - val_loss: 0.0047
Epoch 8/100
5/5 [==============================] - 0s 56ms/step - loss: 0.0049 - val_loss: 0.0047
Epoch 9/100
5/5 [==============================] - 0s 49ms/step - loss: 0.0049 - val_loss: 0.0047
Epoch 10/100
5/5 [==============================] - 0s 54ms/step - loss: 0.0049 - val_loss: 0.0046
Epoch 11/100
5/5 

In [11]:
# Make predictions
y_pred = model.predict(x_test)

3/3 [==============================] - 1s 12ms/step


In [13]:
# Calculate mean squared error
mse = np.mean((y_pred - y_test) ** 2)
print('Mean Squared Error:', mse)

Mean Squared Error: 0.0025161399470066278
